# AutoML

## Imports

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import mltable
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl, Input
import json
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.core.exceptions import HttpResponseError
import pandas as pd
from pathlib import Path
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AzureBlobDatastore



## Grabbing Workspace Configuration

In [ ]:


# Load the Azure ML workspace from config.json
workspace = Workspace.from_config()

# Extract workspace details
subscription_id = workspace.subscription_id
resource_group = workspace.resource_group
workspace_name = workspace.name

# Display the workspace details
print("\n✅ Azure ML Workspace Loaded Successfully\n")
print(f"🔑 Subscription ID : {subscription_id}")
print(f"📦 Resource Group  : {resource_group}")
print(f"🧠 Workspace Name  : {workspace_name}\n")


✅ Azure ML Workspace Loaded Successfully

🔑 Subscription ID : 03571d3d-f675-492f-8346-8b04c78ed80f
📦 Resource Group  : student-performance-rg
🧠 Workspace Name  : student-performance-ws



## Setting Up ML Client

In [ ]:
# 🔐 Authenticate and connect to Azure ML workspace
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

# ✅ Confirm connection
print(f"✅ Connected to Azure ML workspace: {ml_client.workspace_name}")

✅ Connected to Azure ML workspace: student-performance-ws


## Retrieve Registered Data Asset

In [5]:
try:
    data_asset = ml_client.data.get(name=data_asset_name, version=data_asset_version)
    data_path = data_asset.path

    print(f"✅ Retrieved data asset: '{data_asset.name}' (version {data_asset.version})")
    print(f"📍 Data path: {data_path}")
except HttpResponseError as e:
    print(f"❌ Azure ML error: {e.message}")
    raise
except Exception as e:
    print(f"❌ Failed to retrieve data asset: {e}")
    raise

✅ Retrieved data asset: 'student-math' (version 1)
📍 Data path: azureml://subscriptions/03571d3d-f675-492f-8346-8b04c78ed80f/resourcegroups/student-performance-rg/workspaces/student-performance-ws/datastores/studentmathdatastore/paths/student-mat.csv


## Load CSV into DataFrame from URI

In [6]:
data_uri = data_asset.path
print(f"📍 Data URI: {data_uri}")

try:
    # Try to load it directly into pandas
    df = pd.read_csv(data_uri, delimiter=csv_delimiter, encoding=csv_encoding)
    print("✅ DataFrame loaded successfully from URI.")
    display(df.head())
except Exception as e:
    print(f"❌ Failed to load data from URI: {e}")

📍 Data URI: azureml://subscriptions/03571d3d-f675-492f-8346-8b04c78ed80f/resourcegroups/student-performance-rg/workspaces/student-performance-ws/datastores/studentmathdatastore/paths/student-mat.csv
✅ DataFrame loaded successfully from URI.


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
